# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,malanje,-9.5402,16.3410,63.86,98,92,1.90,AO,1735247946
1,1,port-aux-francais,-49.3500,70.2167,41.25,83,84,42.19,TF,1735247947
2,2,vilyuchinsk,52.9306,158.4028,7.07,85,92,5.88,RU,1735247950
3,3,vorgashor,67.5833,63.9500,21.79,96,100,13.42,RU,1735247952
4,4,aasiaat,68.7098,-52.8699,0.68,93,100,8.55,GL,1735247953


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
#%%capture --no-display

# Configure the map plot
humidityPoints = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    size= "Humidity",
    scale = 1,
    color = "City", 
    alpha = 0.7, 
    tiles = "OSM",
    frame_height = 500,
    frame_width = 800,
    hover_cols = ["City", "Humidity"]

)

# Display the map
humidityPoints 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [147]:
# Narrow down cities that fit criteria and drop any results with null values... 
    #A max temperature lower than 86 degrees(F) but higher than 20 degrees(F)
    #Wind speed less than 2 m/s
    #Zero cloudiness

new_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 20.0) & (city_data_df["Max Temp"] < 86.0) & (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
new_city_df = new_city_df.dropna()

# Display sample data
print(len(city_data_df))
print(len(new_city_df))
new_city_df.head()

568
18


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,peace river,56.2501,-117.2860,23.31,73,0,0.00,CA,1735248018
116,116,palmer,42.1584,-72.3287,33.53,60,0,0.00,US,1735248092
117,117,kolonia,50.9333,6.9500,39.90,93,0,2.30,DE,1735248093
121,121,port elizabeth,-33.9180,25.5701,76.32,89,0,3.44,ZA,1735248045
134,134,bethel,41.3712,-73.4140,34.93,56,0,1.01,US,1735248114


### Step 3: Create a new DataFrame called `hotel_df`.

In [149]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
58,peace river,CA,56.2501,-117.2860,73,
116,palmer,US,42.1584,-72.3287,60,
117,kolonia,DE,50.9333,6.9500,93,
121,port elizabeth,ZA,-33.9180,25.5701,89,
134,bethel,US,41.3712,-73.4140,56,
154,kashmar,IR,35.2383,58.4656,45,
156,saiha,IN,22.4833,92.9667,60,
222,diphu,IN,25.8333,93.4333,86,
273,las vegas,US,36.1750,-115.1372,31,
274,lata,PT,40.1629,-8.3327,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [153]:
# Set parameters to search for a hotel
radius = 1000000
params = {
    "categories": "accomodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{long}, {radius}"
    params["bias"] = f"circle:{lat},{long}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
peace river - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
kolonia - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
kashmar - nearest hotel: No hotel found
saiha - nearest hotel: No hotel found
diphu - nearest hotel: No hotel found
las vegas - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
kallar kahar - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
godda - nearest hotel: No hotel found
florence - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
ravar - nearest hotel: No hotel found
livingston - nearest hotel: No hotel found
kothi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,peace river,CA,56.2501,-117.2860,73,No hotel found
116,palmer,US,42.1584,-72.3287,60,No hotel found
117,kolonia,DE,50.9333,6.9500,93,No hotel found
121,port elizabeth,ZA,-33.9180,25.5701,89,No hotel found
134,bethel,US,41.3712,-73.4140,56,No hotel found
154,kashmar,IR,35.2383,58.4656,45,No hotel found
156,saiha,IN,22.4833,92.9667,60,No hotel found
222,diphu,IN,25.8333,93.4333,86,No hotel found
273,las vegas,US,36.1750,-115.1372,31,No hotel found
274,lata,PT,40.1629,-8.3327,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [159]:
# I tried my best but couldn't find any hotels despite multiple filter changes

#Drops duplicates as some values in the hover_cols printed twice 
hotel_df = hotel_df.drop_duplicates(subset=["Lng", "Lat"])

# Configure the map plot
mapPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True, 
    size= "Humidity",
    scale = 1,
    color = "City", 
    alpha = 0.7, 
    tiles = "OSM",
    frame_height = 500,
    frame_width = 800,
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"]
    
)

# Display the map
mapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)